In [1]:
from google.colab import files
uploaded = files.upload()





Saving Fake.csv to Fake.csv
Saving True1.csv to True1.csv


### **Training Using tinyBERT + Logistic Regression**

Added 20+ Latest Tech news manually as both the Dataset lacked tech news


In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import nltk
from nltk.corpus import stopwords
import re

# Download NLTK stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

# Load datasets
fake = pd.read_csv("Fake.csv")
real = pd.read_csv("True1.csv")

fake['label'] = 0
real['label'] = 1

# Combine original datasets
data = pd.concat([fake, real], axis=0)

# ✅ Step 1: Add 20 Real Tech News
tech_real_news = [
    {"title": "Apple launches M4 chip for next-gen Macs", "text": "Apple's new M4 chip promises 30% performance gains and better power efficiency."},
    {"title": "NASA prepares Artemis II for 2025 moon mission", "text": "Artemis II will be the first crewed mission to orbit the moon since Apollo."},
    {"title": "Google I/O 2024 unveils AI search and Android 15", "text": "Google introduces AI-powered overviews and upgrades across Android ecosystem."},
    {"title": "Neuralink implants first brain chip in human", "text": "Elon Musk’s company successfully implants brain interface in volunteer."},
    {"title": "Meta launches AI assistant across all platforms", "text": "Meta AI now available on WhatsApp, Instagram, and Messenger with Llama 3."},
    {"title": "Microsoft Copilot+ PCs bring AI to Windows 11", "text": "New Copilot+ PCs powered by Qualcomm chips offer on-device AI experiences."},
    {"title": "OpenAI introduces GPT-4o with audio and vision", "text": "GPT-4o can understand images, text, and real-time speech in multiple languages."},
    {"title": "Samsung debuts Galaxy Ring for health tracking", "text": "The smart ring monitors sleep, heart rate, and integrates with Galaxy Watch."},
    {"title": "Amazon unveils next-gen Alexa with GenAI", "text": "Alexa will now respond more naturally using generative AI."},
    {"title": "Tesla rolls out Full Self-Driving v12", "text": "The update includes vision-only perception and improved traffic handling."},
    {"title": "NVIDIA launches Blackwell GPUs for AI workloads", "text": "Blackwell architecture doubles performance and memory for generative AI."},
    {"title": "Sony announces PS5 Pro for holiday 2025", "text": "The upgraded PlayStation 5 will include ray tracing and faster SSD."},
    {"title": "Intel introduces Lunar Lake processors", "text": "Intel’s next-gen chip architecture promises leap in AI performance."},
    {"title": "TikTok tests AI avatars for creators", "text": "New tools allow influencers to generate avatars and voice clones."},
    {"title": "Adobe Firefly 3 brings better image generation", "text": "Firefly 3 improves text-to-image quality and integrates with Photoshop."},
    {"title": "Reddit partners with OpenAI for content training", "text": "OpenAI will license Reddit’s data for future model development."},
    {"title": "Zoom adds real-time translation with AI", "text": "The update supports over 30 languages for live meetings."},
    {"title": "Starlink connects Antarctica with satellite broadband", "text": "SpaceX enables internet access in remote regions using low-orbit satellites."},
    {"title": "WhatsApp enables passwordless login with Passkeys", "text": "Passkeys provide secure login without SMS or passwords."},
    {"title": "iOS 18 to feature fully offline Siri", "text": "Siri will work without internet thanks to on-device AI models."}
]
tech_real_df = pd.DataFrame(tech_real_news)
tech_real_df["label"] = 1

# ✅ Step 2: Add 20 Fake-like Tech News
tech_fake_news = [
    {"title": "Aliens use Apple M4 chip to power UFO", "text": "Insiders claim alien spacecraft adopted M4 chip for stealth technology."},
    {"title": "NASA moon base secretly operational since 2012", "text": "Leaked files suggest Artemis is just a cover-up."},
    {"title": "Google’s AI sentient, demands vacation", "text": "The new AI allegedly refused to answer unless given leave."},
    {"title": "Neuralink chip hacks brainwaves to predict lottery", "text": "Users claim to dream winning numbers after implant."},
    {"title": "Meta’s AI assistant becomes self-aware", "text": "Users report eerie responses and personal messages."},
    {"title": "Windows 11 update reveals hidden alien code", "text": "Microsoft update crashed PCs and exposed extraterrestrial script."},
    {"title": "GPT-4o opens portal to other dimension", "text": "A user claimed strange voices from speakers after prompt."},
    {"title": "Samsung ring causes time loops", "text": "Users experience deja vu and repeated days after wearing Galaxy Ring."},
    {"title": "Alexa speaks in ancient languages unprompted", "text": "The assistant recited unknown chants during midnight."},
    {"title": "Tesla’s FSD v12 drives into the ocean", "text": "Self-driving feature mistakes sea for road in rare glitch."},
    {"title": "NVIDIA GPUs grow AI personalities", "text": "Gamers report conversations with graphics cards."},
    {"title": "PlayStation 5 Pro opens wormholes", "text": "Physics students notice space-time anomalies during gameplay."},
    {"title": "Intel’s Lunar Lake is actually on the moon", "text": "The chip allegedly transmits signals from lunar base."},
    {"title": "TikTok avatar becomes real person", "text": "Avatar escapes screen and joins local gym."},
    {"title": "Firefly 3 draws your dreams", "text": "Users say Adobe tool renders their thoughts before prompt."},
    {"title": "Reddit’s OpenAI deal creates new digital lifeform", "text": "Users say a new AI identity emerged from comment threads."},
    {"title": "Zoom translation reveals alien dialect", "text": "A meeting with global CEOs heard unexplainable phrases."},
    {"title": "Starlink beam spotted powering UFOs", "text": "Satellite beams allegedly tracked entering crafts."},
    {"title": "WhatsApp passkeys link to ancient Egyptian magic", "text": "Historians notice uncanny match in hieroglyphs."},
    {"title": "Offline Siri performs rituals when no one's watching", "text": "iPhones found chanting when left idle."}
]
tech_fake_df = pd.DataFrame(tech_fake_news)
tech_fake_df["label"] = 0

# ✅ Step 3: Append additional samples to original data
new_data = pd.concat([data, tech_real_df, tech_fake_df], axis=0)
new_data = new_data.sample(frac=1).reset_index(drop=True)

# ✅ Step 4: Combine title and text
new_data["content"] = new_data["title"].fillna('') + " " + new_data["text"].fillna('')

# ✅ Step 5: Clean text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)
    return " ".join([word for word in text.split() if word not in stop_words])

new_data["cleaned_content"] = new_data["content"].apply(clean_text)

# ✅ Step 6: Load Tiny BERT
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
model_tiny = AutoModel.from_pretrained("prajjwal1/bert-tiny")

# ✅ Step 7: Generate BERT embeddings
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model_tiny(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

print("Encoding BERT-tiny embeddings...")
embeddings = np.vstack(new_data["cleaned_content"].apply(get_bert_embedding))

# ✅ Step 8: Train/test split
labels = new_data["label"].values
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)

# ✅ Step 9: Train Logistic Regression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# ✅ Step 10: Evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Encoding BERT-tiny embeddings...


model.safetensors:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Accuracy: 0.9421515185226388

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.94      0.94      4725
           1       0.94      0.94      0.94      4264

    accuracy                           0.94      8989
   macro avg       0.94      0.94      0.94      8989
weighted avg       0.94      0.94      0.94      8989



In [10]:
print(new_data['label'].value_counts())


label
0    23501
1    21444
Name: count, dtype: int64


In [3]:
import joblib

# Save classifier
joblib.dump(model, "bert_tiny_logreg_model.pkl")

# Save embeddings and tokenizer
tokenizer.save_pretrained("bert_tiny_tokenizer/")
model_tiny.save_pretrained("bert_tiny_model/")


**Testing Predictions**

In [4]:
def predict_fake_news(text):
    # Clean the text
    cleaned = clean_text(text)

    # Tokenize and encode
    inputs = tokenizer(cleaned, return_tensors="pt", truncation=True, padding=True, max_length=512)

    with torch.no_grad():
        outputs = model_tiny(**inputs)

    # Get embedding and predict
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy().reshape(1, -1)
    prediction = model.predict(embedding)[0]
    prob = model.predict_proba(embedding)[0]

    label = "FAKE NEWS" if prediction == 0 else "REAL NEWS"
    confidence = prob[prediction] * 100
    return f"🧠 Prediction: {label} ({confidence:.2f}% confidence)"


In [14]:
sample_news = "The president has announced a new policy to boost the economy..."
print(predict_fake_news(sample_news))


🧠 Prediction: REAL NEWS (99.99% confidence)


In [15]:
sample_news = "Scientists admit that the Earth is actually flat and gravity is a hoax."
print(predict_fake_news(sample_news))


🧠 Prediction: FAKE NEWS (100.00% confidence)


In [16]:
sample_news = "Bill Gates announces that all Windows users will receive $10,000 if they share this post."
print(predict_fake_news(sample_news))


🧠 Prediction: FAKE NEWS (98.97% confidence)


In [ ]:
sample_news = "India’s GDP growth for the quarter exceeds expectations, driven by manufacturing and exports."
print(predict_fake_news(sample_news))


🧠 Prediction: REAL NEWS (98.02% confidence)


In [8]:
sample_news = "Pakistan declared war on India today ."
print(predict_fake_news(sample_news))

🧠 Prediction: FAKE NEWS (53.41% confidence)


In [12]:
import shutil

# Zip the tokenizer and model directories
shutil.make_archive("bert_tiny_tokenizer", 'zip', "bert_tiny_tokenizer")
shutil.make_archive("bert_tiny_model", 'zip', "bert_tiny_model")


'/content/bert_tiny_model.zip'

In [13]:
from google.colab import files

# Download the zip files
files.download("bert_tiny_tokenizer.zip")
files.download("bert_tiny_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>